In [20]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, BatchNormalization

In [7]:
def load_data(data_path):
    data = []
    labels = []
    emotions = {'angry_05': 0, 'boredom': 1, 'calm_02': 2, 'disgust_07': 3, 'fearful_06': 4, 'happy_03': 5, 'neutral_01': 6, 'pain': 7, 'sad_04': 8, 'suprised_08': 9} # Duygulara göre etiketler
    for emotion in emotions.keys():
        emotion_path = os.path.join(data_path, emotion)
        for filename in os.listdir(emotion_path):
            if filename.endswith('.wav'):
                filepath = os.path.join(emotion_path, filename)
                feature = extract_feature(filepath)
                data.append(feature)
                labels.append(emotions[emotion])
    return np.array(data), np.array(labels)

In [21]:
def extract_feature(file_path, mfcc=True, chroma=True, mel=True):
    audio_data, sample_rate = librosa.load(file_path)
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(y=audio_data, sr=sample_rate).T,axis=0)
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(y=audio_data, sr=sample_rate).T,axis=0)
    if mfcc:
        mfcc = np.mean(librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40).T,axis=0)
    return np.hstack((mfcc,chroma,mel))

In [22]:
data, labels = load_data('D:/yapayzekafinal/ses')
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


d:\Anaconda\Lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


In [23]:
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

In [24]:
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(LSTM(128, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(LSTM(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [25]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_test, y_test))

# Modelin performansını değerlendirme

Epoch 1/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 37s 355ms/step - accuracy: 0.1287 - loss: 2.2990 - val_accuracy: 0.1707 - val_loss: 2.2670
Epoch 2/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 28s 332ms/step - accuracy: 0.1665 - loss: 2.2693 - val_accuracy: 0.0509 - val_loss: 2.2821
Epoch 3/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 28s 330ms/step - accuracy: 0.1886 - loss: 2.1947 - val_accuracy: 0.1063 - val_loss: 2.2942
Epoch 4/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 27s 326ms/step - accuracy: 0.2699 - loss: 1.9080 - val_accuracy: 0.0734 - val_loss: 2.1200
Epoch 5/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 27s 326ms/step - accuracy: 0.2813 - loss: 1.8326 - val_accuracy: 0.2111 - val_loss: 1.8457
Epoch 6/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 28s 328ms/step - accuracy: 0.2815 - loss: 1.7861 - val_accuracy: 0.2904 - val_loss: 1.7440
Epoch 7/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 27s 320ms/step - accuracy: 0.2996 - loss: 1.7606 - val_accuracy: 0.0943 - val_loss: 2.1528
Epoch 8/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 27s 322ms/step - accuracy: 0.2855 - loss: 1.7563 - 

KeyboardInterrupt: 